## DPR Reader

### Trying example script

In [1]:
from transformers import DPRReader, DPRReaderTokenizer
tokenizer = DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')
model = DPRReader.from_pretrained('facebook/dpr-reader-single-nq-base')
encoded_inputs = tokenizer(
        questions=["What is love ?"],
        titles=["Haddaway"],
        texts=["'What Is Love' is a song recorded by the artist Haddaway"],
        return_tensors='pt'
    )

outputs = model(**encoded_inputs)
# start_logits = outputs.start_logits
# end_logits = outputs.end_logits
# relevance_logits = outputs.relevance_logits

Some weights of DPRReader were not initialized from the model checkpoint at facebook/dpr-reader-single-nq-base and are newly initialized: ['span_predictor.encoder.bert_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
passage=["'What Is Love' is a song recorded by the artist Haddaway"]

In [23]:
tokens = tokenizer.convert_ids_to_tokens(list(encoded_inputs['input_ids'].numpy()[0]))

In [28]:
import pandas as pd
import numpy as np


predicted_span = ' '.join(tokens[np.argmax(outputs[0].detach().numpy()[0]) : np.argmax(outputs[1].detach().numpy()[0]) + 1])
print(predicted_span)

a song


### Trying on our text

In [29]:
import os
import re
import glob
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [30]:
def absoluteFilePaths(directory):
    path = []
    files = []
    for dirpath, dirname, filenames in os.walk(directory):
        for f in filenames:
            if not os.path.basename(dirpath).startswith('.'):
                path.append(dirpath)
                files.append(f)
            
    return path, files

In [31]:
question = "How many position were opened in March"
true_answer = "100000"

# question = "How many new people hired by amazon"
# true_answer = "100000"

# question = "What are dominant sequence transduction models based on"
# true_answer = " complex recurrent or convolutional neural networks that include an encoder and a decoder"

# question = "What is attention mechanism"
# true_answer = "The attention mechanism is a part of a neural architecture that enables to dynamically highlight relevant features of the input data, which, in NLP, is typically a sequence of textual elements. It can be applied directly to the raw input or to its higher level representation."

# question = "What is quantum entanglement"
# true_answer = "Quantum Entanglement allows qubits that are separated by incredible distances to interact with each other instantaneously (not limited to the speed of light)."

# question = "What are the applications of Face Swapping"
# true_answer = "Face swapping has a number of compelling applications in video compositing, transfiguration in portraits, and especially in  identity  protection  as  it  can  replace  faces  in  photographs by ones from a collection of stock images"

In [32]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [33]:
DIRECTORY = os.getcwd()
locations, documents = absoluteFilePaths(os.path.join(DIRECTORY, 'Google', 'research'))
paths = [os.path.join(loc, doc) for loc, doc in zip(locations, documents)]

In [34]:
paths

['C:\\Users\\hiteshsom\\Documents\\nlp_document_finder\\Google\\research\\2019-Annual-Report-pages-1-5.pdf']

In [35]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [36]:
contents = []
for path in paths:
    if path.endswith('.pdf'):
        contents.append(convert_pdf_to_txt(path))
    else: 
        continue

In [37]:
CHUNK_SIZE = 100

In [39]:
contents = [re.sub(r'\n', ' ', content) for content in contents]

### Using DPR model to get QA predict

In [ ]:
tokenizer = DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')
model = DPRReader.from_pretrained('facebook/dpr-reader-single-nq-base')

In [43]:
CHUNK_SIZE = 20

In [46]:
answers = []
paragraphs = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(".")), CHUNK_SIZE):
        paragraphs.append(content.split(".")[i:i+CHUNK_SIZE])
#         print(paragraph)
#         print("\n")
encoded_inputs = tokenizer(
    questions=[question],
    titles=["AMAZON Report", "Beyond Covid", "Leveraging Scale for good"],
    texts=paragraphs,
    return_tensors='pt'
    )
outputs = model(**encoded_inputs)



1it [00:00, ?it/s]


AssertionError: There should be as many titles than texts but got 3 titles and 7 texts.

In [ ]:

encoded_inputs = tokenizer(
        questions=["What is love ?"],
        titles=["Haddaway"],
        texts=["'What Is Love' is a song recorded by the artist Haddaway"],
        return_tensors='pt'
    )

outputs = model(**encoded_inputs)
